# Setup

trust me and ignore it.

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-fo0e0i0z
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-fo0e0i0z
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 3cd943628b5c415585c8ef100f65989f6adc7f75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:00

## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [81.0 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,079 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("cool person")

In [4]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [5]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [6]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [7]:
def imshow(tensor, renderer=None, midpoint=0.0, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=midpoint, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [8]:
line(np.arange(5))

set-up device

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Picking a behaviour!

The first step when trying to reverse engineer a circuit in a model is to identify what capability you want to reverse engineer.

Pick a behaviour you find interesting:

- "1 2 3 4" -> " 5"
- "I, II, III, IV," -> "V"
- "Monday, Tuesday,"->" Wednesday"
- "April, May, June," -> " July"
- "His name is John." -> " He"
- "If today is Monday, tomorrow is" -> " Tuesday"
- "Mary is a great friend, isn’t" -> " she"
- "Paris is the capital of" -> " France"
- "\<div> \<h2> hello world \</h2></" -> "div"
- "[{(2, 3), (4, 6)}, {(9, 8)}" -> "]"
- "I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked" -> " an"
- "After John and Mary went to the store, John gave a bottle of milk to" -> " Mary"
- Choose your own adventure! Try finding behaviours of your own.

Okay now let's load in our model! You can chosse from the following models:
- "gpt2-small"
- "gpt2-medium"
- "gpt2-large"
- "distillgpt2"
- "opt-125m"
- "gpt-neo-125M"
- "pythia-70m"
- "pythia-160m"
- "pythia-410m"

more info on the particular models is [here](https://github.com/neelnanda-io/TransformerLens/blob/main/transformer_lens/model_properties_table.md)

In [10]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    refactor_factored_attn_matrices=True,
    device=device
    )

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


The next step is to verify that the model can actually do the task!

In [11]:
example_prompt = "Mary is a great friend, isn’t"
example_answer = " she"
utils.test_prompt(example_prompt, example_answer, model, prepend_space_to_answer = False)

Tokenized prompt: ['<|endoftext|>', 'Mary', ' is', ' a', ' great', ' friend', ',', ' isn', '�', '�', 't']
Tokenized answer: [' she']


Performance on answer token:
Rank: 0        Logit: 17.41 Prob: 82.67% Token: | she|

Top 0th token. Logit: 17.41 Prob: 82.67% Token: | she|
Top 1th token. Logit: 14.69 Prob:  5.45% Token: | it|
Top 2th token. Logit: 13.51 Prob:  1.68% Token: | he|
Top 3th token. Logit: 13.11 Prob:  1.12% Token: | there|
Top 4th token. Logit: 12.74 Prob:  0.78% Token: | I|
Top 5th token. Logit: 12.72 Prob:  0.76% Token: | we|
Top 6th token. Logit: 12.67 Prob:  0.72% Token: | you|
Top 7th token. Logit: 12.35 Prob:  0.52% Token: | her|
Top 8th token. Logit: 12.23 Prob:  0.47% Token: | this|
Top 9th token. Logit: 12.16 Prob:  0.43% Token: | that|


Ranks of the answer tokens: [(' she', 0)]

Hope it can, otherwise change to a more suitable model.

We now run the model on the prompt and use run_with_cache to get both the logits and a cache of all internal activations for later analysis.

In [12]:
prompt = 'Mary is a great friend, isn’t'
answer = (" she", " he")
answer_tokens = torch.tensor((
    model.to_single_token(answer[0]),
    model.to_single_token(answer[1]),
    )).cuda()
answer_tokens

tensor([673, 339], device='cuda:0')

In [13]:
tokens = model.to_tokens(prompt, prepend_bos=True)
# Move the tokens to the GPU
tokens = tokens.cuda()
# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

We'll later be evaluating how model performance differs upon performing various interventions, so it's useful to have a metric to measure model performance. Our metric here will be the **logit difference**, the difference in logit between the correct answer and the incorrect answer.

In [14]:
def logits_to_ave_logit_diff(logits, answer_tokens):
    # Only the final logits are relevant for the answer
    final_logits = logits[0, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[0] - answer_logits[1]
    return answer_logit_diff

original_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(f"logit difference: {original_logit_diff.item():.4f}")
print(f"probability of correct answer: {torch.exp(original_logit_diff).item():.4f}x")

logit difference: 3.8977
probability of correct answer: 49.2901x


# Brainstorm What's Actually Going On (Optional)

Before diving into running experiments, it's often useful to spend some time actually reasoning about how the behaviour in question could be implemented in the transformer.

You don't have to do this and forming hypotheses after exploration is also reasonable, but I think it's often easier to explore and interpret results with some grounding in what you might find.

Note that often your hypothesis will be wrong in some ways and often be completely off. We're doing science here, and the goal is to understand how the model *actually* works, and to form true beliefs!

# Direct Logit Attribution

We use `model.tokens_to_residual_directions` to map the answer tokens to residual stream direction, and then convert this to a logit difference direction for each batch.

In [ ]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]

To verify that this works, we can apply this to the final residual stream for our cached prompts (after applying LayerNorm scaling) and verify that we get the same answer.

In [ ]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream = cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = cache.apply_ln_to_stack(final_token_residual_stream, layer = -1, pos_slice=-1)

average_logit_diff = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream, logit_diff_directions)/len(prompts)
print("Calculated average logit diff:", average_logit_diff.item())
print("Original logit difference:",logits_to_ave_logit_diff(original_logits, answer_tokens).item())

Final residual stream shape: torch.Size([1, 11, 768])


RuntimeError: ignored

### Logit Lens

We can now decompose the residual stream! First we apply a technique called the [**logit lens**](https://www.alignmentforum.org/posts/AcKRB8wDpdaN6v6ru/interpreting-gpt-the-logit-lens) - this looks at the residual stream after each layer and calculates the logit difference from that. This simulates what happens if we delete all subsequence layers.

In [ ]:
def residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("... d_model, d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(prompt)

In [ ]:
accumulated_residual, labels = cache.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(logit_lens_logit_diffs, x=np.arange(model.cfg.n_layers*2+1)/2, hover_name=labels, title="Logit Difference From Accumulate Residual Stream")

NameError: ignored

We see that the model is utterly unable to do the task until layer 8 and then the performance starts to increase from there in a step fashion with jumps at the attention part of the layer. Also, note that layer 11’s MLP decreases the logit diff.

### Layer Attribution

We can repeat the above analysis but for each layer (this is equivalent to the differences between adjacent residual streams)

In [ ]:
per_layer_residual, labels = cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

We see that attention layers 9, 10, and 11 matter significantly at directly contributing to the logit difference!

### Head Attribution

We can further break down the output of each attention layer into the sum of the outputs of each attention head. Each attention layer consists of 12 heads, which each act independently and additively.

In [ ]:
per_head_residual, labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
imshow(per_head_logit_diffs, labels={"x":"Head", "y":"Layer"}, title="Logit Difference From Each Head")

We see that only a few heads possess significant contributions towards the logit difference - heads L9H7, L10H9, and L11H8 contribute a lot positively (explaining why attention layers 9, 10, and 11 are so important). There are also several heads that matter positively or negatively but less strongly.

### Attention Analysis

We use Anthropic's PySvelte library to visualize the attention patterns! We visualize the top 3 positive heads by direct logit attribution, and show these for the first prompt (as an illustration).

In [ ]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: Optional[ActivationCache]=None,
    local_tokens: Optional[torch.Tensor]=None,
    title: str=""):
    # Heads are given as a list of integers or a single integer in [0, n_layers * n_heads)
    if isinstance(heads, int):
        heads = [heads]
    elif isinstance(heads, list) or isinstance(heads, torch.Tensor):
        heads = utils.to_numpy(heads)
    # Cache defaults to the original activation cache
    if local_cache is None:
        local_cache = cache
    # Tokens defaults to the tokenization of the first prompt (including the BOS token)
    if local_tokens is None:
        # The tokens of the first prompt
        local_tokens = tokens[0]

    labels = []
    patterns = []
    batch_index = 0
    for head in heads:
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    str_tokens = model.to_str_tokens(local_tokens)
    patterns = torch.stack(patterns, dim=-1)
    # Plot the attention patterns
    attention_vis = pysvelte.AttentionMulti(attention=patterns, tokens=str_tokens, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attention_vis.show()

In [ ]:
top_k = 3
top_positive_logit_attr_heads = torch.topk(per_head_logit_diffs.flatten(), k=top_k).indices
visualize_attention_patterns(top_positive_logit_attr_heads, title=f"Top {top_k} Positive Logit Attribution Heads")

They seem to be attending to other tokens like “ is”, “ friend” and “ isn” instead of the subject. So that could mean the processed information about the subject was moved to these tokens in previous heads.

# Activation Patching

### Residual Stream

Lets begin by patching in the residual stream at the start of each layer and for each token position.

We'll first create a corrupted prompt with opposite answer.

In [15]:
corrupted_prompt = 'John is a great friend, isn’t'
corrupted_tokens = model.to_tokens(corrupted_prompt, prepend_bos=True)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
corrupted_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)

ensure both prompts have same number of tokens (very important)

In [16]:
str_tokens = model.to_str_tokens(prompt)
print("Clean Prompt length:", len(str_tokens))
print("Clean Prompt as tokens:", str_tokens)
corrupted_str_tokens = model.to_str_tokens(corrupted_prompt)
print("Corrupted Prompt length:", len(corrupted_str_tokens))
print("Corrupted Prompt as tokens:", corrupted_str_tokens)

Clean Prompt length: 11
Clean Prompt as tokens: ['<|endoftext|>', 'Mary', ' is', ' a', ' great', ' friend', ',', ' isn', '�', '�', 't']
Corrupted Prompt length: 11
Corrupted Prompt as tokens: ['<|endoftext|>', 'John', ' is', ' a', ' great', ' friend', ',', ' isn', '�', '�', 't']


We now intervene on the corrupted run and patch in the clean residual stream at a specific layer and position.

In [17]:
def patch_residual_component(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    clean_cache):
    corrupted_residual_component[:, pos, :] = clean_cache[hook.name][:, pos, :]
    return corrupted_residual_component

def normalize_patched_logit_diff(patched_logit_diff):
    # Subtract corrupted logit diff to measure the improvement, divide by the total improvement from clean to corrupted to normalise
    # 0 means zero change, negative means actively made worse, 1 means totally recovered clean performance, >1 means actively *improved* on clean performance
    return (patched_logit_diff - corrupted_logit_diff)/(original_logit_diff - corrupted_logit_diff)

patched_residual_stream_diff = torch.zeros(model.cfg.n_layers, tokens.shape[1], device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("resid_pre", layer),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_residual_stream_diff[layer, position] = normalize_patched_logit_diff(patched_logit_diff)

In [18]:
prompt_position_labels = [f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(tokens[0]))]
imshow(patched_residual_stream_diff, x=prompt_position_labels, title="Normalized Logit Difference From Patched Residual Stream", labels={"x":"Position", "y":"Layer"})

### Layers

We can apply exactly the same idea, but this time patching in attention or MLP layers.

In [19]:
patched_attn_diff = torch.zeros(model.cfg.n_layers, tokens.shape[1], device="cuda", dtype=torch.float32)
patched_mlp_diff = torch.zeros(model.cfg.n_layers, tokens.shape[1], device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_attn_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("attn_out", layer),
                hook_fn)],
            return_type="logits"
        )
        patched_attn_logit_diff = logits_to_ave_logit_diff(patched_attn_logits, answer_tokens)
        patched_mlp_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("mlp_out", layer),
                hook_fn)],
            return_type="logits"
        )
        patched_mlp_logit_diff = logits_to_ave_logit_diff(patched_mlp_logits, answer_tokens)

        patched_attn_diff[layer, position] = normalize_patched_logit_diff(patched_attn_logit_diff)
        patched_mlp_diff[layer, position] = normalize_patched_logit_diff(patched_mlp_logit_diff)

In [20]:
imshow(patched_attn_diff, x=prompt_position_labels, title="Normalized Logit Difference From Patched Attention Layer", labels={"x":"Position", "y":"Layer"})

In [21]:
imshow(patched_mlp_diff, x=prompt_position_labels, title="Normalized Logit Difference From Patched MLP Layer", labels={"x":"Position", "y":"Layer"})

### Heads

We can refine the above analysis by patching in individual heads!

In [22]:
def patch_head_vector(
    corrupted_head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    position,
    clean_cache):
    corrupted_head_vector[:, position, head_index, :] = clean_cache[hook.name][:, position, head_index, :]
    return corrupted_head_vector

patched_head_z_diff = torch.zeros(model.cfg.n_layers*model.cfg.n_heads, tokens.shape[1], device="cuda", dtype=torch.float32)
for i in range(model.cfg.n_layers*model.cfg.n_heads):
    for position in range(tokens.shape[1]):
        layer = i//model.cfg.n_heads
        head_index = i % model.cfg.n_heads
        hook_fn = partial(patch_head_vector, head_index=head_index, position=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("z", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff[i, position] = normalize_patched_logit_diff(patched_logit_diff)

In [23]:
head_names = [f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]

imshow(patched_head_z_diff, y=head_names, title="Normalized Logit Difference From Patched Head Output", labels={"x":"Position", "y":"Head"})

In [24]:
def patch_head_vector_all_pos(
    corrupted_head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    clean_cache):
    corrupted_head_vector[:, :, head_index, :] = clean_cache[hook.name][:, :, head_index, :]
    return corrupted_head_vector

patched_head_z_diff_all_pos = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector_all_pos, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("z", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff_all_pos[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [25]:
imshow(patched_head_z_diff_all_pos, title="Normalized Logit Difference From Patched Head Output (All Position)", labels={"x":"Head", "y":"Layer"})

### Decomposing Heads

First let's patch in the value vectors, to measure when figuring out what to move is important.

In [26]:
patched_head_v_diff = torch.zeros(model.cfg.n_layers*model.cfg.n_heads, tokens.shape[1], device="cuda", dtype=torch.float32)
for i in range(model.cfg.n_layers*model.cfg.n_heads):
    for position in range(tokens.shape[1]):
        layer = i//model.cfg.n_heads
        head_index = i % model.cfg.n_heads
        hook_fn = partial(patch_head_vector, head_index=head_index, position=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("v", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff[i, position] = normalize_patched_logit_diff(patched_logit_diff)

In [27]:
imshow(patched_head_v_diff, y=head_names, title="Normalized Logit Difference From Patched Head Value", labels={"x":"Position", "y":"Head"})

In [28]:
patched_head_v_diff_all_pos = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector_all_pos, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("v", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff_all_pos[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [29]:
imshow(patched_head_v_diff_all_pos, title="Normalized Logit Difference From Patched Head Value (All Position)", labels={"x":"Head", "y":"Layer"})

It's very Hooked to interpret if we plot a scatter plot against patching head outputs.

In [30]:
scatter(
    x=utils.to_numpy(patched_head_v_diff_all_pos.flatten()),
    y=utils.to_numpy(patched_head_z_diff_all_pos.flatten()),
    xaxis="Value Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name = head_names,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs value patching (All Position)")

After patching the value vectors, we see that the logit difference values are very similar to that of output patching. This means that calculating the value vector is a lot more important for the attention head’s output performance compared to calculating the attention pattern.

Now let's patch in the attention pattern, to measure when figuring out where to move is important.

In [31]:
def patch_head_pattern(
    corrupted_head_pattern: Float[torch.Tensor, "batch head_index dest_pos src_pos"],
    hook,
    head_index,
    position,
    clean_cache):
    corrupted_head_pattern[:, head_index, position, :] = clean_cache[hook.name][:, head_index, position, :]
    return corrupted_head_pattern

patched_head_attn_diff = torch.zeros(model.cfg.n_layers*model.cfg.n_heads, tokens.shape[1], device="cuda", dtype=torch.float32)
for i in range(model.cfg.n_layers*model.cfg.n_heads):
    for position in range(tokens.shape[1]):
        layer = i//model.cfg.n_heads
        head_index = i % model.cfg.n_heads
        hook_fn = partial(patch_head_pattern, head_index=head_index, position=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("attn", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff[i, position] = normalize_patched_logit_diff(patched_logit_diff)

In [32]:
imshow(patched_head_attn_diff, y=head_names, title="Normalized Logit Difference From Patched Head Pattern", labels={"x":"Position", "y":"Head"})

Note the difference in logit diff scale for this and value patching.

In [33]:
def patch_head_pattern_all_pos(
    corrupted_head_pattern: Float[torch.Tensor, "batch head_index query_pos d_head"],
    hook,
    head_index,
    clean_cache):
    corrupted_head_pattern[:, head_index, :, :] = clean_cache[hook.name][:, head_index, :, :]
    return corrupted_head_pattern

patched_head_attn_diff_all_pos = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_pattern_all_pos, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("attn", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff_all_pos[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [34]:
imshow(patched_head_attn_diff_all_pos, title="Normalized Logit Difference From Patched Head Pattern (All Position)", labels={"x":"Head", "y":"Layer"})
scatter(
    x=utils.to_numpy(patched_head_attn_diff_all_pos.flatten()),
    y=utils.to_numpy(patched_head_z_diff_all_pos.flatten()),
    xaxis="Attention Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name = head_names,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs attention patching (All Position)")

Patching attention patterns doesn't seem to affect the head's output much. It presumably means that both the clean prompt and corrupted prompt provide the same information as to where to move information from and to and thus patching doesn’t have any effect.

In [35]:
patched_head_q_diff = torch.zeros(model.cfg.n_layers*model.cfg.n_heads, tokens.shape[1], device="cuda", dtype=torch.float32)
for i in range(model.cfg.n_layers*model.cfg.n_heads):
    for position in range(tokens.shape[1]):
        layer = i//model.cfg.n_heads
        head_index = i % model.cfg.n_heads
        hook_fn = partial(patch_head_vector, head_index=head_index, position=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("q", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_q_diff[i, position] = normalize_patched_logit_diff(patched_logit_diff)

In [36]:
imshow(patched_head_q_diff, y=head_names, title="Normalized Logit Difference From Patched Head Query", labels={"x":"Position", "y":"Head"})

In [37]:
patched_head_q_diff_all_pos = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector_all_pos, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("q", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_q_diff_all_pos[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [38]:
imshow(patched_head_q_diff_all_pos, title="Normalized Logit Difference From Patched Head Query (All Position)", labels={"x":"Head", "y":"Layer"})

It's very Hooked to interpret if we plot a scatter plot against patching head outputs.

In [39]:
scatter(
    x=utils.to_numpy(patched_head_q_diff_all_pos.flatten()),
    y=utils.to_numpy(patched_head_z_diff_all_pos.flatten()),
    xaxis="Query Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name = head_names,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs query patching (All Position)")

Let's patch in the key vectors!

In [40]:
patched_head_k_diff = torch.zeros(model.cfg.n_layers*model.cfg.n_heads, tokens.shape[1], device="cuda", dtype=torch.float32)
for i in range(model.cfg.n_layers*model.cfg.n_heads):
    for position in range(tokens.shape[1]):
        layer = i//model.cfg.n_heads
        head_index = i % model.cfg.n_heads
        hook_fn = partial(patch_head_vector, head_index=head_index, position=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("k", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_k_diff[i, position] = normalize_patched_logit_diff(patched_logit_diff)

In [41]:
imshow(patched_head_k_diff, y=head_names, title="Normalized Logit Difference From Patched Head Key", labels={"x":"Position", "y":"Head"})

In [42]:
patched_head_k_diff_all_pos = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector_all_pos, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks = [(utils.get_act_name("k", layer, "attn"),
                hook_fn)],
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_k_diff_all_pos[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [43]:
imshow(patched_head_k_diff_all_pos, title="Normalized Logit Difference From Patched Head Key (All Position)", labels={"x":"Head", "y":"Layer"})

It's very Hooked to interpret if we plot a scatter plot against patching head outputs.

In [44]:
scatter(
    x=utils.to_numpy(patched_head_k_diff_all_pos.flatten()),
    y=utils.to_numpy(patched_head_z_diff_all_pos.flatten()),
    xaxis="Key Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name = head_names,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs key patching (All Position)")

### Neurons

Let's patch individual neurons!

In [45]:
def patch_neuron(
   corrupted_neuron_vector: Float[torch.Tensor, "batch pos d_mlp"],
   hook,
   neuron,
   clean_cache):
   corrupted_neuron_vector[:, :, neuron] = clean_cache[hook.name][:, :, neuron]
   return corrupted_neuron_vector

imp_mlp = [0, 3, 5, 7, 11] # Important MLP layers

patched_neuron_diff = torch.zeros(len(imp_mlp), model.cfg.d_mlp, device="cuda", dtype=torch.float32)

for i, layer in enumerate(imp_mlp):
   for neuron in range(model.cfg.d_mlp):
       hook_fn = partial(patch_neuron, neuron=neuron, clean_cache=cache)
       patched_logits = model.run_with_hooks(
           corrupted_tokens,
           fwd_hooks = [(utils.get_act_name("pre", layer, "mlp"),
               hook_fn)],
           return_type="logits"
       )
       patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)
       patched_neuron_diff[i,neuron] = normalize_patched_logit_diff(patched_logit_diff)

In [46]:
scatter(
   x=torch.arange(model.cfg.d_mlp).repeat(len(imp_mlp)),
   y=patched_neuron_diff.view(-1),
   xaxis="Neuron",
   yaxis="Patch Improvement",
   caxis="Layer",
   color=[str(x) for x in imp_mlp for _ in range(model.cfg.d_mlp)],
   title="Normalized Logit Difference From Patched Neuron (All Position)")

Okay let's see what these neurons are doing? One handy tool for exploring neurons is [Neuroscope](https://neuroscope.io/)

This process can be automated to some degree, see OpenAI's [work](https://openaipublic.blob.core.windows.net/neuron-explainer/neuron-viewer/index.html#/) on this.

# Visualizing Attention Patterns

Looking at the attention patterns of these heads. Let's take the top 15 heads by output patching (in absolute value) and split it into early, middle and late.

In [47]:
top_k = 15
top_heads_by_output_patch = torch.topk(patched_head_z_diff_all_pos.abs().flatten(), k=top_k).indices
first_mid_layer = 4
first_late_layer = 8
early_heads = top_heads_by_output_patch[top_heads_by_output_patch<model.cfg.n_heads * first_mid_layer]
mid_heads = top_heads_by_output_patch[torch.logical_and(model.cfg.n_heads * first_mid_layer<=top_heads_by_output_patch, top_heads_by_output_patch<model.cfg.n_heads * first_late_layer)]
late_heads = top_heads_by_output_patch[model.cfg.n_heads * first_late_layer<=top_heads_by_output_patch]
visualize_attention_patterns(early_heads, title=f"Top Early Heads")
visualize_attention_patterns(mid_heads, title=f"Top Middle Heads")
visualize_attention_patterns(late_heads, title=f"Top Late Heads")

NameError: ignored

# Backup Heads

If we knock out one of the heads, then there are some backup heads in later layers that *change their behaviour* and do (some of) the job of the original head.

Let's test this! Let's ablate the most important head (head L10H9) on just the final token using a custom ablation hook and then cache all new activations and compared performance. We focus on the final position because we want to specifically ablate the direct logit effect.

In [ ]:
top_head = per_head_logit_diffs.flatten().argmax().item()
top_head_layer = top_head//model.cfg.n_heads
top_head_head = top_head % model.cfg.n_heads
print(f"Top head to ablate: L{top_head_layer}H{top_head_head}")
def ablate_top_head_hook(z: Float[torch.Tensor, "batch pos head_index d_head"], hook):
    z[:, -1, top_head_head, :] = 0
    return z
# Adds a hook into global model state
model.blocks[top_head_layer].attn.hook_z.add_hook(ablate_top_head_hook)
# Runs the model, temporarily adds caching hooks and then removes *all* hooks after running, including the ablation hook.
ablated_logits, ablated_cache = model.run_with_cache(tokens)
print(f"Original logit diff: {original_logit_diff}")
print(f"Post ablation logit diff: {logits_to_ave_logit_diff(ablated_logits, answer_tokens).item()}")
print(f"Direct Logit Attribution of top head: {per_head_logit_diffs.flatten()[top_head].item()}")
print(f"Naive prediction of post ablation logit diff: {original_logit_diff - per_head_logit_diffs.flatten()[top_head].item()}")

We see that naively, ablating the top head should reduce the logit diff by 1.35, from 4.0 to 2.64. **But actually, it only goes down to 3.2!**

As before, we can look at the direct logit attribution of each head after ablating head L10H9 to see what's going on. It's easiest to interpret if plotted as a scatter plot against the initial per head logit difference. We also plot the difference between the original and post-ablation logit diff of each head.

In [ ]:
per_head_ablated_residual, labels = ablated_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_ablated_logit_diffs = residual_stack_to_logit_diff(per_head_ablated_residual, ablated_cache)
per_head_ablated_logit_diffs = per_head_ablated_logit_diffs.reshape(model.cfg.n_layers, model.cfg.n_heads)
imshow(per_head_ablated_logit_diffs, labels={"x":"Head", "y":"Layer"}, title="Post-Ablation Direct Logit Attribution of Heads")
scatter(y=per_head_logit_diffs.flatten(), x=per_head_ablated_logit_diffs.flatten(), hover_name=head_names, range_x=(-3, 3), range_y=(-3, 3), xaxis="Ablated", yaxis="Original", title="Original vs Post-Ablation Direct Logit Attribution of Heads")
imshow((per_head_ablated_logit_diffs- per_head_logit_diffs), labels={"x":"Head", "y":"Layer"}, title="Difference in Post-Ablation Direct Logit Attribution and Original Direct Logit Attribution of Heads")



We can see some minor difference in a few heads! Head L11H1 increases its effect, adding +0.19 to the logit diff, and the head L11H8 decreases its effect, adding -0.12 to the logit diff. Though not sure if it's enough to justify them as backup heads.

# Universality

It is the hypothesis that the same circuits will show up in different models.

Let's see it in action.